### Ej 1

#### 1.1: Corpus

Comenzamos exportando los datos del archivo .tgz

In [1]:
import tarfile

In [2]:
tar = tarfile.open('ap.tgz')

In [3]:
tar.extractall()

Sobre el archivo .txt que contiene los datos de AP en xml aplicamos un parser manual para separar los documentos

In [4]:
docno = []
corpus = []
toggle = 0
with open('ap/ap.txt','r') as f:
    for line in f:
        if line.startswith('<DOCNO>'):
            line_new = line.replace('<DOCNO> ','')
            line_new = line_new.replace(' </DOCNO>\n','')
            docno.append(line_new)
        if toggle:
            corpus.append(line)
            toggle = 0
        if line.startswith('<TEXT>'):
            toggle = 1        
docs = dict(zip(docno, corpus))        

Otra forma sería usar el parser del ElementTree, pero para usarlo correctamente resulta necesario reemplazar todos los caracteres '&' por '&amp;' y agregar un tag que delimite el comienzo y final del archivo

In [6]:
with open('ap/ap.txt','r') as f, open('ap/ap_cleaned.txt','w') as f_new:
    f_new.write('<ROOT>\n')
    for line in f:
        new_line = line.replace('&','&amp;')
        f_new.write(new_line)
    f_new.write('</ROOT>\n')
f_new.close()

Ahora aplicamos el ElementTree

In [7]:
import xml.etree.ElementTree as ET

In [8]:
ap = ET.parse('ap/ap_cleaned.txt')
ap

In [9]:
ap_root = ap.getroot()

In [10]:
ap_dict = dict([
        (doc[0].text.strip(), doc[1].text)
        for doc in ap_root
    ])

Y observamos que de las 2 maneras separamos el xml en la misma cantidad de documentos

In [11]:
len(ap_dict)

2250

In [12]:
len(docs)

2250

#### 1.2: Tamaño vocabulario

In [13]:
import nltk

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HB\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HB\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Tokenizamos y pasamos a minúscula todos los textos.

In [16]:
def tokenize(string):
    'Returns list of tokens'
    # TODO: Cambiar split por: https://stackoverflow.com/questions/15547409/how-to-get-rid-of-punctuation-using-nltk-tokenizer
    words = string.split(' ')  # [Palabra]
    wnl = nltk.WordNetLemmatizer() 
    lemma_words = [wnl.lemmatize(word) for word in words]  # [Palabra]
    retoken = [w 
               for lw in lemma_words
               for w in nltk.word_tokenize(lw) ]
    return retoken

In [245]:
textos_tokenizados = [
    token
    for text in ap_dict.values()
    for token in tokenize(text)
]

In [246]:
print('Tamaño vocabulario: {}'.format(len(set(textos_tokenizados))))

Tamaño vocabulario: 46394


In [247]:
tokens_ordenados = sorted(textos_tokenizados)

In [248]:
from collections import Counter

In [249]:
c = Counter(tokens_ordenados)

In [250]:
c['doctor'] * 5

585